In [ ]:
import pandas as pd
from plotly import express as ex

# MJ2383 Computer Lab 3
This lab aims to provide an inside view on what a supply curve is and how it can be generated while using OSeMOSYS. Reminder, a supply curve is a graphical representation of the law of supply. It slopes upward because quantity supplied rises as price rises, with other things constant.

In this lab, we'll be using OSeMOSYS.

In [ ]:
!glpsol -d super_simple_model.txt -m osemosys.txt > osemosys.log

In [ ]:
%ls results

In [ ]:
demand = pd.read_csv('results/Demand.csv').groupby(by=['TIMESLICE', 'FUEL', 'YEAR']).sum()
ex.line(demand.reset_index(), x='YEAR', y='VALUE', color='FUEL')